In [145]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.cluster import KMeans
from sklearn.gaussian_process.kernels import Matern, RBF

import plotly.express as px

import plotly.graph_objects as go

from scipy.linalg import toeplitz, block_diag

from spe.mse_estimator import ErrorComparer
from spe.data_generation import gen_rbf_X, gen_matern_X, create_clus_split, gen_cov_mat
from spe.relaxed_lasso import RelaxedLasso
from spe.estimators import kfoldcv, kmeanscv, better_test_est_split, cp_linear_train_test, cp_relaxed_lasso_train_test, cp_rf_train_test, bag_kfoldcv, bag_kmeanscv
from spe.forest import BlurredForest

import os

In [176]:
niter = 100

n=30**2
p=30
s=30

k=5

alpha = 0.05
delta = 1.

n_estimators = 100
max_depth = 6

lambd = .4
nboot = 100

snr = 0.4

noise_kernel = 'matern'
noise_length_scale = 1.
noise_nu = .5

X_kernel = 'rbf'
X_length_scale = 5.
X_nu = 2.5

idx = -1

savedir='~'

In [177]:
# noise_kernel = kernel
# noise_length_scale = length_scale
# noise_nu = nu

if X_kernel is None:
    X_kernel = kernel
if X_length_scale is None:
    X_length_scale = noise_length_scale
if X_nu is None:
    X_nu = noise_nu


In [178]:
# models = RelaxedLasso(lambd=lambd)
# models = LinearRegression(fit_intercept=False)
models = BlurredForest(n_estimators=n_estimators,  
                  max_depth=max_depth, 
                  bootstrap_type='blur')

ests = [better_test_est_split,
        cp_rf_train_test, 
#         cp_relaxed_lasso_train_test, 
#         cp_linear_train_test, 
        bag_kfoldcv, 
        bag_kmeanscv]
est_kwargs = [{'alpha': None, 'full_refit':True},
              {'use_trace_corr': True, 
               'full_refit':True},
#               {'alpha': alpha, 
#                'use_trace_corr': True, 
#                'full_refit': True,
#                'nboot': nboot}, 
#               {},
              {'k': k},
              {'k': k}]

In [179]:
if not os.path.exists(os.path.expanduser(savedir)):
    os.makedirs(os.path.expanduser(savedir))

params = pd.DataFrame({'niter': niter,
                       'n': n, 
                       'p': p, 
                       's': s,
                       'k': k,
                       'snr': snr,
                       'lambd': lambd,
                       'alpha': alpha,
                       'delta': delta,
                       'nboot': nboot,
                       'nk': noise_kernel, 
                       'nls': noise_length_scale, 
                       'nn': noise_nu, 
                       'xk': X_kernel,
                       'xls': X_length_scale,
                       'xn': X_nu}, index=[idx])
params.to_csv(os.path.expanduser(savedir + 'params.csv'))
dffp = os.path.expanduser(savedir + "err_df.csv")
# barfp = os.path.expanduser(savedir + 'barchart.jpeg')

In [180]:
err_cmp = ErrorComparer()

In [181]:
nx = ny = int(np.sqrt(n))
xs = np.linspace(0, 10, nx)
ys = np.linspace(0, 10, ny)
c_x, c_y = np.meshgrid(xs, ys)
c_x = c_x.flatten()
c_y = c_y.flatten()
coord = np.stack([c_x, c_y]).T

In [182]:
if noise_kernel == 'rbf':
    Sigma_t = gen_cov_mat(c_x, c_y, RBF(length_scale=noise_length_scale))
elif noise_kernel == 'matern':
    Sigma_t = gen_cov_mat(c_x, c_y, Matern(length_scale=noise_length_scale, nu=noise_nu))
else:
    Sigma_t = np.eye(n)
    
Sigma_t = delta*Sigma_t + (1-delta)*np.eye(n)

if noise_kernel == 'rbf' or noise_kernel == 'matern':
    Chol_t = np.linalg.cholesky(Sigma_t)
else:
    Chol_t = np.eye(n)

In [183]:
if X_kernel == 'rbf':
    X = gen_rbf_X(c_x, c_y, p)
elif X_kernel == 'matern':
    X = gen_matern_X(c_x, c_y, p, length_scale=X_length_scale, nu=X_nu)
else:
    X = np.random.randn(n,p)

beta = np.zeros(p)
idx = np.random.choice(p,size=s)
beta[idx] = np.random.uniform(-1,1,size=s)

In [191]:
Sigma_t

array([[1.00000000e+00, 7.08342471e-01, 5.01749056e-01, ...,
        1.16454371e-06, 9.18573059e-07, 7.21354153e-07],
       [7.08342471e-01, 1.00000000e+00, 7.08342471e-01, ...,
        1.46951116e-06, 1.16454371e-06, 9.18573059e-07],
       [5.01749056e-01, 7.08342471e-01, 1.00000000e+00, ...,
        1.84526167e-06, 1.46951116e-06, 1.16454371e-06],
       ...,
       [1.16454371e-06, 1.46951116e-06, 1.84526167e-06, ...,
        1.00000000e+00, 7.08342471e-01, 5.01749056e-01],
       [9.18573059e-07, 1.16454371e-06, 1.46951116e-06, ...,
        7.08342471e-01, 1.00000000e+00, 7.08342471e-01],
       [7.21354153e-07, 9.18573059e-07, 1.16454371e-06, ...,
        5.01749056e-01, 7.08342471e-01, 1.00000000e+00]])

In [184]:
(test_err,
#  rnd_err,
 trc_err,
 kfcv_err,
 spcv_err) = err_cmp.compare(models,
                             ests,
                             est_kwargs,
                             niter=niter,
                             n=n,
                             p=p,
                             s=p,
                             snr=snr, 
                             X=X,
                             beta=beta,
                             coord=coord,
                             Chol_t=Chol_t,
                             Chol_s=None,
                             tr_idx=None,
                             fair=False,
                             tr_frac=1./3,
                             )

0
0.32
10
20
30
40
50
60
70
80
90


In [185]:
risk = test_err.mean()
risk_kfcv = kfcv_err.mean()
risk_spcv = spcv_err.mean()
# risk_rnd = rnd_err.mean()
risk_trc = trc_err.mean()
# risk, risk_kfcv, risk_spcv, risk_lin

In [186]:
save_df = pd.DataFrame({#'Rand_WR': (rnd_err.T),
#                         'Trace_WR': (trc_err.T),
                        'GenCp': (trc_err.T),
                        'KFCV': (kfcv_err.T),
                        'SPCV': (spcv_err.T),
                        'Test': (test_err.T)})
save_df.to_csv(dffp)

In [187]:
df = pd.DataFrame({#'Rand_WR': (rnd_err.T),
#                    'Trace_WR': (trc_err.T),
                   'GenCp': (trc_err.T),
                   'KFCV': (kfcv_err.T),
                   'SPCV': (spcv_err.T)})

In [188]:
(df/risk).mean()

GenCp    1.011815
KFCV     0.364991
SPCV     0.984114
dtype: float64

In [189]:
df.var(), (df/risk).var()

(GenCp    0.000040
 KFCV     0.000003
 SPCV     0.000161
 dtype: float64,
 GenCp    0.016893
 KFCV     0.001360
 SPCV     0.067588
 dtype: float64)

In [190]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df.columns, 
    y=(df/risk).mean(),
    marker_color=px.colors.qualitative.Plotly,
    text=np.around((df/risk).mean(),3),
    textposition='outside',
    error_y=dict(
        type='data',
        color='black',
        symmetric=False,
        array=(df/risk).quantile(.75) - (df/risk).mean(),
        arrayminus=(df/risk).mean() - (df/risk).quantile(.25))
#         array=(df/risk).mean() + (df/risk).std(),
#         arrayminus=(df/risk).mean() - (df/risk).std())
))
# fig.add_trace(go.Bar(
#     name='Experimental',
#     x=['GenCp', 'KFCV', 'SPCV'], y=(df/risk).mean(),
#     error_y=dict(type='data', array=[1, 2])
# ))
# fig.update_layout(barmode='group')
fig.add_hline(y=1., line_color='red')
fig.update_layout(
#     title=f"CRL_\u03B1{alpha}_\u03B4{delta}_snr{snr}_nk{noise_kernel}_nls{noise_length_scale}_nv{noise_nu}_Xk{X_kernel}_Xls{X_length_scale}_Xv{X_nu}",
    title=f"Random Forest: Correction vs 5 Fold-CV",
    xaxis_title="Method",
    yaxis_title="Relative MSE",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"
#     )
)
barfp = os.path.expanduser(savedir + fig.layout.title['text'] + ".jpeg")
fig.write_image(os.path.expanduser(barfp))
fig.show()